# Verify Credential

This notebook illustrates how a VC signed using a BIP 322 signature could be parsed and verified by a verifier.

## Notebook Author

* Will Abramson
* [Legendary Requirements](http://legreq.com/)
* Contact: will@legreq.com

## Acknowledgements

This work was funded by Ryan Grant and [Digital Contract Design](https://contract.design/). Thanks also go to Joe Andrieu, Kalle Alm, Pieter Wuille and Jimmy Song for engaging with and supporting various aspects of this work.

## Imports

In [1]:
cd ..

/home/will/work/LegendaryRequirements/clients/dcd/bip0322-sigs


In [45]:
import copy

from pyld import jsonld

from buidl.helper import str_to_bytes, sha256

from src.message import verify_message

## Copy Signed VC from issue_credential Notebook

**YOU SHOULD COPY THE SIGNED VC PRODUCED IN the [issue_credential](http://localhost:8888/notebooks/vc/issue_credential.ipynb) NOTEBOOK**

This is the VC we will attempt to verify in this notebook.

In [13]:
signed_vc = {'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1', 'https://w3id.org/security/suites/ed25519-2020/v1'], 'id': 'http://example.edu/credentials/1872', 'type': ['VerifiableCredential', 'AlumniCredential'], 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t', 'issuanceDate': '2010-01-01T19:23:24Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': [{'value': 'Example University', 'lang': 'en'}, {'value': "Exemple d'Université", 'lang': 'fr'}]}}, 'proof': {'type': 'Ed25519Signature2020', 'created': '2022-08-22 15:01:27.800288', 'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1', 'proofPurpose': 'assertionMethod', 'proofValue': 'AkcwRAIgOvSMw2n2FtRGFQ2XhchjZ0smBoR66tlqdSzKJRlyf90CIFMEfKEc78oMQlzefGnuJtOcpe6hq0AUh5Sm9fO7kkEaASEDyIIldNfHBVrefH3VmN1a9H5yNUtW4/oeLCw88CX2zzE='}}
signed_vc

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://w3id.org/security/suites/ed25519-2020/v1'],
 'id': 'http://example.edu/credentials/1872',
 'type': ['VerifiableCredential', 'AlumniCredential'],
 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t',
 'issuanceDate': '2010-01-01T19:23:24Z',
 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21',
  'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1',
   'name': [{'value': 'Example University', 'lang': 'en'},
    {'value': "Exemple d'Université", 'lang': 'fr'}]}},
 'proof': {'type': 'Ed25519Signature2020',
  'created': '2022-08-22 15:01:27.800288',
  'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1',
  'proofPurpose': 'assertionMethod',
  'proofValue': 'AkcwRAIgOvSMw2n2FtRGFQ2XhchjZ0smBoR66tlqdSzKJRlyf90CIFMEfKEc78oMQlzefGnuJtOcpe6hq0AUh5Sm9fO7kkEaASEDyIIldNfHBVrefH3VmN1a9H5yNUtW4/oeLCw88CX2zzE='}}

## Data Integrity Proof Verification Algorithm

Steps taken from [here](https://w3c.github.io/vc-data-integrity/#proof-verification-algorithm):

1. Get the public key by dereferencing its URL identifier in the proof node of the default graph of signed document. 
    - Confirm that the unsigned data document that describes the public key specifies its controller and that its controllers's URL identifier can be dereferenced to reveal a bi-directional link back to the key. 
    - Ensure that the key's controller is a trusted entity before proceeding to the next step.
2. Let document be a copy of signed document.
3. Remove any proof nodes from the default graph in document and save it as proof.
4. Generate a canonicalized document by canonicalizing document according to the canonicalization algorithm (e.g. the URDNA2015 [RDF-DATASET-C14N] algorithm).
5. Create a value tbv that represents the data to be verified, and set it to the result of running the Create Verify Hash Algorithm, passing the information in proof.
6. Pass the proofValue, tbv, and the public key to the proof algorithm (e.g. JSON Web Proof using RSASSA-PKCS1-v1_5 algorithm). Return the resulting boolean value.


## 1. Get the public key by dereferencing its URL identifier in the proof node of the default graph of signed document. 

In the case of a VC, we need to resolve the issuer identifier (typically a DID) to retrieve the DID document.


In [14]:
issuer_did_btcr = signed_vc["issuer"]

proof = signed_vc["proof"]
verificationMethod = proof["verificationMethod"]

print("Need to be able to deference : ", verificationMethod)

Need to be able to deference :  did:btcr:xyv2-xzpq-q9wa-p7t#vm-1


### 1.1. First resolve the DID to get the corresponding DID document

**Note:** this is mocked in these notebooks. We KNOW that the issuer did resolves to a specific DID document defined in the below cell.

In [48]:
resolved_did_doc = {
    "@context": ["https://w3id.org/did/v0.11", "https://w3id.org/btcr/v1"],
    "id": "did:btcr:xyv2-xzpq-q9wa-p7t",
    "verificationMethod": [
       {
            "id": "did:btcr:xyv2-xzpq-q9wa-p7t#satoshi",
            "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
            "type": "EcdsaSecp256k1VerificationKey2019",
            "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
        },
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vckey-0",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "EcdsaSecp256k1VerificationKey2019",
             "publicKeyBase58": "owh12LKNuphe97teJTZKQTKNewSVTwjHcskPbq34epCY"
         },
    ##########################      EXAMPLE       #################################
        ### What are the resolution rules that parse the Bitcoin Tx at ref xyv2-xzpq-q9wa-p7t
        ### To populate this verificationMethod into the didDocument.
        ### E.g. scriptPubKey from UTX0 at index 1 in tx at ref xyv2-xzpq-q9wa-p7t
        {
             "id": "did:btcr:xyv2-xzpq-q9wa-p7t#vm-1",
             "controller": "did:btcr:xyv2-xzpq-q9wa-p7t",
             "type": "BIP322VerificationAddress2022",
             "address": "bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck"
        }
    
    ###############################################################################
    ],
    "authentication": ["#satoshi"],
    "assertionMethod": ["#vckey-0","#vm-1"]
}


### 1.2. Parse Resolved DID document for "publicKey" defined in proof

Note: in the case of a BIP322 verificationMethod the publicKey is actually a bitcoin address.

In [49]:
bip322VerificationAddress = None
for vm in resolved_did_doc["verificationMethod"]:
    if vm["id"] == verificationMethod:
        bip322VerificationAddress = vm["address"]
        break;
        
print("Bitcoin address to use in BIP 322 Verification : ", bip322VerificationAddress)

Bitcoin address to use in BIP 322 Verification :  bc1qz52z3pe9fg3qxv9n6yhxgj7rcn8wsvpq56v9ck


## 2. Let document be a copy of signed document

Essentially copy the signed_vc object

In [17]:
document = copy.deepcopy(signed_vc)


## 3. Remove any proof nodes from the default graph in document and save it as proof.

In [18]:
proof = document["proof"]
del document["proof"]
print(document)
print(proof)

{'@context': ['https://www.w3.org/2018/credentials/v1', 'https://www.w3.org/2018/credentials/examples/v1', 'https://w3id.org/security/suites/ed25519-2020/v1'], 'id': 'http://example.edu/credentials/1872', 'type': ['VerifiableCredential', 'AlumniCredential'], 'issuer': 'did:btcr:xyv2-xzpq-q9wa-p7t', 'issuanceDate': '2010-01-01T19:23:24Z', 'credentialSubject': {'id': 'did:example:ebfeb1f712ebc6f1c276e12ec21', 'alumniOf': {'id': 'did:example:c276e12ec21ebfeb1f712ebc6f1', 'name': [{'value': 'Example University', 'lang': 'en'}, {'value': "Exemple d'Université", 'lang': 'fr'}]}}}
{'type': 'Ed25519Signature2020', 'created': '2022-08-22 15:01:27.800288', 'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1', 'proofPurpose': 'assertionMethod', 'proofValue': 'AkcwRAIgOvSMw2n2FtRGFQ2XhchjZ0smBoR66tlqdSzKJRlyf90CIFMEfKEc78oMQlzefGnuJtOcpe6hq0AUh5Sm9fO7kkEaASEDyIIldNfHBVrefH3VmN1a9H5yNUtW4/oeLCw88CX2zzE='}


## 4. Generate a canonicalized document by canonicalizing document according to the canonicalization algorithm.

In [19]:
normalized_document = jsonld.normalize(
    document, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
print(normalized_document)

<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n0 .
<did:example:c276e12ec21ebfeb1f712ebc6f1> <http://schema.org/name> _:c14n1 .
<did:example:ebfeb1f712ebc6f1c276e12ec21> <http://schema.org/alumniOf> <did:example:c276e12ec21ebfeb1f712ebc6f1> .
<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://example.org/examples#AlumniCredential> .
<http://example.edu/credentials/1872> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://www.w3.org/2018/credentials#VerifiableCredential> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#credentialSubject> <did:example:ebfeb1f712ebc6f1c276e12ec21> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuanceDate> "2010-01-01T19:23:24Z"^^<http://www.w3.org/2001/XMLSchema#dateTime> .
<http://example.edu/credentials/1872> <https://www.w3.org/2018/credentials#issuer> <did:btcr:xyv2-xzpq-q9wa-p7t> .



## 5. Create a value tbv that represents the data to be verified, and set it to the result of running the Create Verify Hash Algorithm, passing the information in proof.

## Create Verify Hash Algoritm

1. Let options be a copy of input options.
2. If the proofValue parameter, such as jws, exists in options, remove the entry.
3. If created does not exist in options, add an entry with a value that is an [ISO8601] combined date and time string containing the current date and time accurate to at least one second, in Universal Time Code format. For example: 2017-11-13T20:21:34Z.
4. Generate output by:
    * Creating a canonicalized options document by canonicalizing options according to the canonicalization algorithm (e.g. the URDNA2015 [RDF-DATASET-C14N] algorithm).
    * Hash canonicalized options document using the message digest algorithm (e.g. SHA-256) and set output to the result.
    * Hash canonicalized document using the message digest algorithm (e.g. SHA-256) and append it to output.



#### 1. Let options be a copy of input options

Note: it is not at all clear from the spec what input_options should be.Commented on issues already tracking this here - https://github.com/w3c/vc-data-integrity/issues/16

For now I will be using the proof options that define how the proof will be created.

In [27]:
options = {
    'proof': copy.deepcopy(proof)
}
options['@context'] = document['@context']
options

{'proof': {'type': 'Ed25519Signature2020',
  'created': '2022-08-22 15:01:27.800288',
  'verificationMethod': 'did:btcr:xyv2-xzpq-q9wa-p7t#vm-1',
  'proofPurpose': 'assertionMethod',
  'proofValue': 'AkcwRAIgOvSMw2n2FtRGFQ2XhchjZ0smBoR66tlqdSzKJRlyf90CIFMEfKEc78oMQlzefGnuJtOcpe6hq0AUh5Sm9fO7kkEaASEDyIIldNfHBVrefH3VmN1a9H5yNUtW4/oeLCw88CX2zzE='},
 '@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1',
  'https://w3id.org/security/suites/ed25519-2020/v1']}

#### 2. If the proofValue parameter, such as jws, exists in options, remove the entry.

In [31]:
if 'proofValue' in options['proof']:
    del options['proof']['proofValue']


#### 3. If created does not exists, add it

It already exists

#### 4. Generate output

In [32]:
canonicalized_options = jsonld.normalize(
    options, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
print(canonicalized_options)

_:c14n0 <http://purl.org/dc/terms/created> "2022-08-22 15:01:27.800288"^^<http://www.w3.org/2001/XMLSchema#dateTime> _:c14n1 .
_:c14n0 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <https://w3id.org/security#Ed25519Signature2020> _:c14n1 .
_:c14n0 <https://w3id.org/security#proofPurpose> <https://w3id.org/security#assertionMethod> _:c14n1 .
_:c14n0 <https://w3id.org/security#verificationMethod> <did:btcr:xyv2-xzpq-q9wa-p7t#vm-1> _:c14n1 .
_:c14n2 <https://w3id.org/security#proof> _:c14n1 .



In [41]:
result = sha256(str_to_bytes(canonicalized_options))

In [42]:
result += sha256(str_to_bytes(normalized_document))

In [43]:
result

b'\x88\xe8\x90\x87\r\xcd\xa1q\xd1c\xd8\x11\xb5]V\x80\xf23\xa8\xc2\x10O/H\x0e9xR7%\x9fz\xbbF\x03B\xb9\\V\x02#,\xdbQRh\xd8\x08;b\xda\xc6\x85<,F\x8b\x95\xb3\x89u\xbc\xc5\xc2'

In [44]:
tbv = result

## 6. Pass the proofValue, tbv, and the public key to the proof algorithm. Return the resulting boolean value.

In [52]:
proof['proofValue']

'AkcwRAIgOvSMw2n2FtRGFQ2XhchjZ0smBoR66tlqdSzKJRlyf90CIFMEfKEc78oMQlzefGnuJtOcpe6hq0AUh5Sm9fO7kkEaASEDyIIldNfHBVrefH3VmN1a9H5yNUtW4/oeLCw88CX2zzE='

In [53]:
verified = verify_message(bip322VerificationAddress, proof['proofValue'], tbv)

mismatch between length and consumed bytes 31 vs 4
mismatch between length and consumed bytes 60 vs 20


In [55]:
print("VC is verified : ", verified)

VC is verified :  True
